<div style="text-align: center;">
<span style="color:#C1D3FE;font-size: 32px; font-weight: bold;">Notebook de Análise: "Originadora" de 05/08</span>
<br> <span style="color:#FFD1DC;"> Feito por Lucas Andrade entre 07 e 11 de Agosto de 2025 </span>
<br> <span style="color:#FFF5BA;"> revisado por Felipe Bastos


<span style="color:#C1D3FE;">Porto Real Asset</span>

## <span style="color:#AEE5F9;">Análise de Carteira de Originadores

Este notebook realiza uma análise completa da carteira de recebíveis, consolidando dados de diferentes fontes, realizando verificações de qualidade, calculando métricas de risco e, por fim, gerando um relatório HTML interativo. Feito por Lucas Andrade, com auxílio de Felipe Bastos

### <span style="color:#AEE5F9;"> Bibliotecas e Configurações Iniciais

In [1]:
# =============================================================================
# Bibliotecas   ===============================================================
# =============================================================================
import pandas as pd
import numpy as np
import os
from scipy.optimize import brentq
import base64
import os

from IPython.display import display

pd.options.display.max_columns = 100
pd.options.display.max_rows = 200

##### <span style="color:#CFFFE5;">NOVIDADE versão 1.02</span>

In [2]:
DIAS_ATRASO_DEFINICAO_VENCIDO = 30

### <span style="color:#AEE5F9;">  Leitura e Preparação dos Dados
<span style="color: #FFB3B3; font-size: 15px; font-weight: bold;">
  ATENÇÃO: REDIFINIR AQUI OS PATHS
</span>

Defino os caminhos dos arquivos de entrada, carrego os dados, uno as duas fontes (`StarCard.xlsx` e `Originadores.xlsx`) usando a coluna `CCB` como chave e realizamos uma limpeza inicial, tratando colunas monetárias e de data.

In [3]:
# =============================================================================
# LER DADOS   =================================================================
# =============================================================================

#! PATHS ----------------------------------------------------------------------
#! ----------------------------------------------------------------------------
# Dados IN : 
path_starcard = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\StarCard.xlsx'
path_originadores = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\Originadores.xlsx'
caminho_feriados = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\feriados_nacionais.xls'

logo_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\images\logo_inv.png'

#SAÍDA LOCAL:
output_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output'
#! ----------------------------------------------------------------------------

print(f"Lendo arquivo principal: {path_starcard}")
df_starcard = pd.read_excel(path_starcard)

print(f"Lendo arquivo de detalhe: {path_originadores}")

# ?<obs> essas foram deduções próprias, q dei minha própria interpretação
cols_originadores = [
    'CCB', 'Prazo', 'Valor Parcela', 'Valor IOF', 'Valor Liquido Cliente',
    'Data Primeiro Vencimento', 'Data Último Vencimento', 'Data de Inclusão',
    'CET Mensal', 'Taxa CCB', 'Produto', 'Tabela', 'Promotora',
    'Valor Split Originador', 'Valor Split FIDC', 'Valor Split Compra de Divida',
    'Taxa Originador Split', 'Taxa Split FIDC'
]
df_originadores = pd.read_excel(path_originadores, usecols=cols_originadores)

# aproveitei que o CCB parece repetir em ambos os lados, pra dar um LEFT JOIN 
#!---------------------------------------------
#TODO :: verificar se essa leitura está correta
# fiz essa verificação que deveria ser sufieciente ao meu entendimento
#!---------------------------------------------
print("Unindo as duas fontes de dados...")
print("Verificando a unicidade da chave 'CCB' em df_originadores...")
if not df_originadores['CCB'].is_unique:
    print("[WARNING] A coluna 'CCB' não é única em Originadores.xlsx. Isso causa duplicação de linhas!")
    #** mostrar os duplicados pra investigr
    duplicados = df_originadores[df_originadores.duplicated(subset='CCB', keep=False)]
    print("CCBs duplicados :")
    display(duplicados.sort_values('CCB'))
else:
    print("'CCB' é uma chave única. A junção tá segura.")



df_merged = pd.merge(df_starcard, df_originadores, on='CCB', how='left', suffixes=('', '_orig'))

print("Iniciando limpeza e preparação dos dados...")

# Aqui eu renomeei para funcionar no script anterior
df_merged = df_merged.rename(columns={
    'Data Referencia': 'DataGeracao',
    'Data Aquisicao': 'DataAquisicao',
    'Data Vencimento': 'DataVencimento',
    'Status': 'Situacao',
    'PDD Total': 'PDDTotal',
    'Valor Nominal': 'ValorNominal',
    'Valor Presente': 'ValorPresente',
    'Valor Aquisicao': 'ValorAquisicao',
    'ID Cliente': 'SacadoID', # obs: nao especifica o doc
    'Pagamento Parcial': 'PagamentoParcial'
})

# remove 'R$ ' -->>> vira float #*(note que os valores vem assim em StarCard)
cols_monetarias = ['ValorAquisicao', 'ValorNominal', 'ValorPresente', 'PDDTotal']
for col in cols_monetarias:
    if df_merged[col].dtype == 'object':
        df_merged[col] = df_merged[col].astype(str).str.replace('R$', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).str.strip()
        df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')

# cols de data
cols_data = ['DataGeracao', 'DataAquisicao', 'DataVencimento', 'Data de Nascimento']
for col in cols_data:
    df_merged[col] = pd.to_datetime(df_merged[col], errors='coerce')

# ? df_final2 Criado AQUI <<<<
df_final2 = df_merged.copy()
# Libera memória
del df_starcard, df_originadores, df_merged

print("Leitura, junção e limpeza concluídas.")
print(f"DataFrame final com {df_final2.shape[0]} linhas e {df_final2.shape[1]} colunas.")

Lendo arquivo principal: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\StarCard.xlsx
Lendo arquivo de detalhe: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\Originadores.xlsx
Unindo as duas fontes de dados...
Verificando a unicidade da chave 'CCB' em df_originadores...
'CCB' é uma chave única. A junção tá segura.
Iniciando limpeza e preparação dos dados...
Leitura, junção e limpeza concluídas.
DataFrame final com 248413 linhas e 39 colunas.


### <span style="color:#AEE5F9;">  Colunas Auxiliares

Crio aqui colunas derivadas que capturam informações importantes, como:
- **_ValorLiquido**: Valor Presente descontado do PDD.
- **_ValorVencido**: Valor Presente de parcelas já vencidas.
- **_MuitosContratos**: Flag para clientes com 3 ou mais contratos.
- **_MuitosEntes**: Flag para clientes com contratos em 3 ou mais convênios.
- **_IdadeCliente**: Idade do cliente calculada na data de geração do relatório.

CHANGELOG: <span style="color:#CFFFE5;">ATUALIZADO na versão 1.02</span>

In [4]:
# =============================================================================
#  Colunas Auxiliares =======================================================
# =============================================================================

df_final2['_ValorLiquido'] = df_final2['ValorPresente'] - df_final2['PDDTotal']
df_final2['_ValorVencido'] = (df_final2['DataVencimento'] <= df_final2['DataGeracao']).astype('int') * df_final2['ValorPresente']

#* [NOVIDADE] ..............................................................................
# ID PARCELAS individuais que estão vencidas conforme a nossa regra.
print(f"Passo 1: Identificando parcelas com {DIAS_ATRASO_DEFINICAO_VENCIDO} ou mais dias de atraso.")
dias_de_atraso = (df_final2['DataGeracao'] - df_final2['DataVencimento']).dt.days
df_final2['_ParcelaVencida_Flag'] = ((dias_de_atraso >= DIAS_ATRASO_DEFINICAO_VENCIDO)).astype(int)
# IDtodos os CCBs  que contêm PELO MENOS UMA parcela vencida.
print("Passo 2: Identificando todos os contratos que possuem ao menos uma parcela vencida.")
contratos_com_parcela_vencida = df_final2.groupby('CCB')['_ParcelaVencida_Flag'].max()
# Filtramos para obter uma lista apenas dos CCBs que de fato estão "contaminados".
lista_ccbs_vencidos = contratos_com_parcela_vencida[contratos_com_parcela_vencida == 1].index
# flag final a nível de CONTRATO.
print("Passo 3: Marcando todas as parcelas de um contrato vencido com a flag final.")
df_final2['_ContratoVencido_Flag'] = df_final2['CCB'].isin(lista_ccbs_vencidos).astype(int)
#*..........................................................................................


# uso 'SacadoID', já que nao tem cpf
sacado_contratos = df_final2.groupby('SacadoID')['CCB'].nunique() # vou usar SacadoID pra achar os sacados com muitos contratos
k = 3
mask_contratos = sacado_contratos >= k
sacado_contratos_alto = sacado_contratos[mask_contratos].index
df_final2['_MuitosContratos'] = df_final2['SacadoID'].isin(sacado_contratos_alto).astype(str)

sacados_entes = df_final2.groupby('SacadoID')['Convênio'].nunique() # muitos entes com sacadoid dnv
k2 = 3
mask_entes = sacados_entes >= k2
sacados_entes_alto = sacados_entes[mask_entes].index
df_final2['_MuitosEntes'] = df_final2['SacadoID'].isin(sacados_entes_alto).astype(str)

#* NOVIDADE: idade do cliente
if 'Data de Nascimento' in df_final2.columns and 'DataGeracao' in df_final2.columns:
    df_final2['_IdadeCliente'] = ((df_final2['DataGeracao'] - df_final2['Data de Nascimento']).dt.days / 365.25).astype(int)
    print("Coluna '_IdadeCliente' criada.")

print("Criação de colunas auxiliares concluída.")

Passo 1: Identificando parcelas com 30 ou mais dias de atraso.
Passo 2: Identificando todos os contratos que possuem ao menos uma parcela vencida.
Passo 3: Marcando todas as parcelas de um contrato vencido com a flag final.
Coluna '_IdadeCliente' criada.
Criação de colunas auxiliares concluída.


### <span style="color:#AEE5F9;"> PDD e Vencidos

PDD e o percentual de títulos vencidos, quebrando a análise pelas diversas variáveis categóricas para identificar os segmentos de maior risco.

In [5]:
# =============================================================================
# analise do PDD e Vencidos   ===============================================>>
# (por variável categórica)   ===============================================>>
# =============================================================================
# PDD e a inadimplência por diversas categorias.
cat_cols = [
    'Situacao', 'Cedente', 'PagamentoParcial',
    '_MuitosContratos', '_MuitosEntes', 'Convênio', 'Originador', 'Produto',
    'UF', 'CAPAG', 'Promotora'
]

cat_cols = [col for col in cat_cols if col in df_final2.columns] # tiro cols que nao estejam no df 

# PDD -----------------------------------------------------------------------------------
print("--- Análise de PDD por Categoria ---")
pdd_ref = (1 - df_final2['_ValorLiquido'].sum() / df_final2['ValorPresente'].sum()) * 100
print(f"PDD de Referência (Total): {pdd_ref:.2f}%\n")

for col in cat_cols:
    print(f"Análise de PDD por '{col}':")
    aux_ = df_final2.groupby(col)[['_ValorLiquido', 'ValorPresente']].sum() / 1e6
    aux_['%PDD'] = (1 - aux_['_ValorLiquido'] / aux_['ValorPresente']) * 100
    display(aux_.sort_values('ValorPresente', ascending=False).head(20))

# Vencidos --------------------------------------------------------------------------
print("\n" + "="*80 + "\n")
print("--- Análise de Vencidos por Categoria ---")
venc_ref = (df_final2['_ValorVencido'].sum() / df_final2['ValorPresente'].sum()) * 100
print(f"Percentual de Vencidos de Referência (Total): {venc_ref:.2f}%\n")

for col in cat_cols:
    print(f"Análise de Vencidos por '{col}':")
    aux_ = df_final2.groupby(col)[['_ValorVencido', 'ValorPresente']].sum() / 1e6
    aux_['%Vencido'] = (aux_['_ValorVencido'] / aux_['ValorPresente']) * 100
    display(aux_.sort_values('%Vencido', ascending=False).head(20))

--- Análise de PDD por Categoria ---
PDD de Referência (Total): 23.40%

Análise de PDD por 'Situacao':


,_ValorLiquido,ValorPresente,%PDD
Situacao,,,
A vencer,24.165965,30.588039,20.995377
Vencido,2.611387,4.376060,40.325620
Previsto,0.154540,0.196854,21.495355


Análise de PDD por 'Cedente':


,_ValorLiquido,ValorPresente,%PDD
Cedente,,,
BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,26.931892,35.160954,23.403979


Análise de PDD por 'PagamentoParcial':


...

,_ValorVencido,ValorPresente,%Vencido
UF,,,
GO,2.859188,18.340010,15.589891
RN,0.078199,0.551887,14.169328
PE,0.083260,0.718529,11.587607
MA,0.047473,0.499191,9.510032
SP,1.213036,12.907580,9.397860
PR,0.020613,0.238650,8.637238
CE,0.055201,1.149046,4.804074
PA,0.019090,0.756060,2.524936


Análise de Vencidos por 'CAPAG':


,_ValorVencido,ValorPresente,%Vencido
CAPAG,,,
C,3.358749,24.189972,13.884884
A,0.827552,7.999844,10.344599
N.D.,0.129760,2.015493,6.438119
B,0.059999,0.955646,6.278370


Análise de Vencidos por 'Promotora':


,_ValorVencido,ValorPresente,%Vencido
Promotora,,,
MTT NEGOCIOS LTDA,0.670273,2.184963,30.676620
SOLAR CONSULTORIA E SERVIÇOS LTDA,0.030838,0.105866,29.129421
SOLUÇÕES ASSESSORIA - ALEXSANDRA GROSSER LTDA,0.001156,0.004493,25.728683
Yeshuah & Figueiredo Soluções Fin. Ltda - GEICEL HENRIQUE PEREIRA DE SOUSA E SILVA,0.000508,0.001976,25.723486
GERAR PARTICIPAÇAO E INTERMEDIAÇAO,0.014865,0.058010,25.624068
X CONNECT CONSULTORIA LTDA,0.007975,0.032161,24.798241
GL CREDITOS INTELIGENTES,0.002702,0.011137,24.265391
REAL CREDITO EMPRESTIMO CONSIGNADO,0.037840,0.176913,21.388959
02081977109 Danielleferrazdamaia,0.135417,0.644157,21.022320


### <span style="color:#AEE5F9;"> Verificações de Consistência Adicionais

Algumas verificações focadas na lógica do negócio, como a presença de clientes em múltiplos convênios e a consistência entre datas e valores.

## Verificações de Consistência

#### Verificação igual ao felipe (normaliza com valor máximo)

In [6]:
#%%
# =============================================================================
# CÉLULA DE GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS (AGING)
# =============================================================================
# Descrição: Este script utiliza a lógica de análise de performance baseada no
# "mês de pico". Ele calcula o volume nominal de cada mês como um percentual
# do mês de maior volume nominal da história, usando a visão de "aging".

import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

# --- CONFIGURAÇÕES INICIAIS ---

# Verifica se o DataFrame principal (df_final2) existe.
if 'df_final2' not in locals() or df_final2.empty:
    raise NameError("O DataFrame 'df_final2' não foi encontrado ou está vazio. Por favor, execute as células anteriores primeiro.")

# Adiciona uma coluna para permitir a análise da carteira total
df_report = df_final2.copy()
df_report['_total_carteira'] = 'Carteira Consolidada'
df_report['MesVencimento'] = pd.to_datetime(df_report['DataVencimento']).dt.to_period('M')

# Define os segmentos que queremos analisar
segmentos_para_analise = {
    'Carteira Consolidada': '_total_carteira',
    'Cedentes': 'Cedente',
    'Originadores': 'Originador',
    'Promotoras': 'Promotora',
    'Produtos': 'Produto',
    'Convênios': 'Convênio',
    'Situação': 'Situacao',
    'UF': 'UF',
    'CAPAG': 'CAPAG',
    'Pagamento Parcial': 'PagamentoParcial',
    'Tem Muitos Contratos':'_MuitosContratos',
    'Tem Muitos Entes':'_MuitosEntes'
}

# Define a data de referência
ref_date_obj = df_report['DataGeracao'].max().date()

# Define o caminho de saída
output_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output'
os.makedirs(output_path, exist_ok=True)


# --- FUNÇÕES AUXILIARES ---

def convert_period_to_relative_month(period: pd.Period, ref_date: date) -> str:
    """Converte um objeto Period para um formato de mês relativo (ex: M-1, M+0)."""
    venc_month = period.to_timestamp().to_pydatetime()
    ref_month = datetime(ref_date.year, ref_date.month, 1)
    delta = relativedelta(venc_month, ref_month)
    months_diff = delta.years * 12 + delta.months
    return f"M{months_diff:+}"

# --- FUNÇÃO PRINCIPAL DE GERAÇÃO DA TABELA ---

def generate_performance_heatmap(df_input: pd.DataFrame, segment_column: str, segment_friendly_name: str, ref_date: date) -> str:
    """
    Gera uma tabela HTML de performance de vencimentos para um segmento.
    """
    print(f"---> Processando segmento: {segment_friendly_name}...")
    
    # Lógica para pular segmentos com apenas uma entrada
    unique_entries = df_input[segment_column].dropna().unique()
    if len(unique_entries) == 1 and segment_column != '_total_carteira':
        entry_name = unique_entries[0]
        print(f"     [INFO] Segmento '{segment_friendly_name}' tem apenas uma entrada: '{entry_name}'. Gerando mensagem simples.")
        message_html = f"""
        <div class="table-container">
            <h3 class="table-title">Análise por: {segment_friendly_name}</h3>
            <p class="single-entry-message">Em '{segment_friendly_name}' temos apenas uma entrada: <strong>{entry_name}</strong>. A análise detalhada deste item é refletida na tabela 'Carteira Consolidada'.</p>
        </div>
        """
        return message_html

    # 1. Criação da Tabela Dinâmica com Valor Nominal
    report_abs = df_input.pivot_table(
        index=segment_column,
        columns="MesVencimento",
        values="ValorNominal",
        fill_value=0,
        aggfunc='sum'
    )
    if report_abs.empty: return ""

    # 2. Cálculo de Pico Dinâmico
    peak_value = report_abs.max().max()
    if peak_value == 0: return ""
    print(f"     Pico de Valor Nominal para este segmento: R$ {peak_value:,.2f}")

    # 3. Cálculo dos Totais de VP e VL
    summary_totals = df_input.groupby(segment_column)[['_ValorLiquido', 'ValorPresente']].sum()
    summary_totals.rename(columns={'_ValorLiquido': 'Valor Líquido Total (R$)', 'ValorPresente': 'Valor Presente Total (R$)'}, inplace=True)

    # 4. Cálculo da Performance
    report_performance = (1 - (report_abs / peak_value)) * 100
    report_performance["Total Nominal (R$)"] = report_abs.sum(axis=1)
    
    # Junta os totais de VP e VL
    report_performance = report_performance.join(summary_totals)
    
    report_performance.sort_values(by="Total Nominal (R$)", ascending=False, inplace=True)
    report_performance.index.name = segment_friendly_name
    report_performance.reset_index(inplace=True)

    # 5. Lógica de Renomeação e Ordenação de Colunas
    period_cols = [c for c in report_performance.columns if isinstance(c, pd.Period)]
    period_to_relative_map = {p: convert_period_to_relative_month(p, ref_date) for p in period_cols}
    relative_to_html_map = {
        relative: f"{relative}<br><span class='sub-header'>{period.strftime('%Y-%m')}</span>"
        for period, relative in period_to_relative_map.items()
    }
    report_performance.rename(columns=period_to_relative_map, inplace=True)
    
    relative_month_cols = [c for c in report_performance.columns if isinstance(c, str) and c.startswith('M')]
    
    # [ALTERADO] Inclui o M+1 na análise para ver adiantamentos
    months_to_show = [c for c in relative_month_cols if int(c.replace('M','').replace('+','')) <= 1]
    
    sort_key = lambda col: int(col.replace('M', '').replace('+', ''))
    # Inverte a ordem das colunas de mês
    sorted_relative_cols = sorted(months_to_show, key=sort_key, reverse=True)
    
    # Adiciona as colunas de VP e VL no início da ordem
    final_cols_order = [
        segment_friendly_name, 
        'Valor Presente Total (R$)', 
        'Valor Líquido Total (R$)'
    ] + sorted_relative_cols + ["Total Nominal (R$)"]
    
    # Garante que apenas colunas existentes sejam selecionadas
    final_cols_order = [col for col in final_cols_order if col in report_performance.columns]
    report_final = report_performance[final_cols_order]

    report_final.rename(columns=relative_to_html_map, inplace=True)
    sorted_html_cols = [relative_to_html_map.get(c, c) for c in sorted_relative_cols]

    # 6. Estilização e Geração do HTML
    def format_br_percent(val):
        if pd.isna(val): return ""
        return f"{val:.2f}%".replace('.', ',')
        
    def format_br_number(val):
        if pd.isna(val): return ""
        return f"{val:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')

    styler = (
        report_final.style
        .set_table_attributes('class="dataframe"')
        .set_properties(**{"padding": "8px", "text-align": "right", "min-width": "80px"})
        .set_table_styles([
            {"selector": "th, td", "props": [("border", "1px solid #ccc")]},
            {"selector": "th", "props": [("background-color", "#163f3f"), ("color", "#FFFFFF"), ("text-align", "center"), ("font-weight", "bold"), ("line-height", "1.3")]},
            {"selector": "thead th", "props": [("position", "sticky"), ("top", "0"), ("z-index", "1")]},
            {"selector": "tbody td:first-child", "props": [("position", "sticky"), ("left", "0"), ("background-color", "#f9f9f9"), ("text-align", "left"), ("font-weight", "bold"), ("white-space", "nowrap")]},
            {"selector": "thead th:first-child", "props": [("position", "sticky"), ("left", "0"), ("z-index", "2")]}
        ], overwrite=False)
        .format({
            **{col: format_br_percent for col in sorted_html_cols}, 
            **{"Total Nominal (R$)": format_br_number, 
               'Valor Presente Total (R$)': format_br_number, 
               'Valor Líquido Total (R$)': format_br_number}
        })
        .background_gradient(cmap="RdYlGn", subset=sorted_html_cols, vmin=0, vmax=100, axis=None)
        .background_gradient(cmap="Greys", subset=["Total Nominal (R$)", 'Valor Presente Total (R$)', 'Valor Líquido Total (R$)'], axis=None)
        .hide(axis="index")
    )
    
    header_tabela = f"""
    <div class="table-container">
        <h3 class="table-title">Performance de Vencimentos por: {segment_friendly_name}</h3>
        {styler.to_html()}
    </div>
    """
    return header_tabela


# --- BLOCO PRINCIPAL DE EXECUÇÃO ---

print("\n" + "="*80)
print("INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS")
print(f"Data de referência para os dados: {ref_date_obj.strftime('%d/%m/%Y')}")
print("="*80)

html_tabelas = []
for nome_amigavel, nome_coluna in segmentos_para_analise.items():
    tabela_html = generate_performance_heatmap(df_report, nome_coluna, nome_amigavel, ref_date_obj)
    if tabela_html:
        html_tabelas.append(tabela_html)

# --- MONTAGEM DO ARQUIVO HTML FINAL ---

if html_tabelas:
    html_css = """
    <style>
        body { font-family: "Segoe UI", Arial, sans-serif; background-color: #f4f4f4; margin: 0; padding: 20px; }
        .main-container { max-width: 95%; margin: auto; background-color: #ffffff; padding: 25px; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
        .report-header { text-align: center; border-bottom: 2px solid #163f3f; padding-bottom: 15px; margin-bottom: 30px; }
        .report-header h1 { color: #163f3f; margin: 0; }
        .report-header p { color: #555; font-size: 1.1em; }
        .table-container { margin-bottom: 50px; }
        .table-title { color: #0e5d5f; border-bottom: 2px solid #76c6c5; padding-bottom: 8px; margin-bottom: 15px; }
        .dataframe { border-collapse: collapse; border-spacing: 0; width: 100%; font-size: 0.9em; }
        .dataframe td.na { background-color: transparent !important; }
        .single-entry-message {
            padding: 20px;
            background-color: #f8f9fa;
            border-left: 5px solid #76c6c5;
            margin-top: 15px;
            font-size: 1.1em;
            color: #333;
        }
        .sub-header {
            font-size: 0.8em;
            font-weight: normal;
            color: #dddddd;
        }
        footer {
            margin-top: 40px;
            padding-top: 20px;
            border-top: 1px solid #ddd;
            font-size: 0.9em;
            color: #777;
            text-align: center;
        }
    </style>
    """
    
    footer_text = """
    <p><strong>Nota sobre a Metodologia:</strong> Este relatório utiliza a lógica de análise de performance baseada no "mês de pico". 
    Ele calcula o volume nominal de cada mês como um percentual do mês de maior volume nominal da história para aquele segmento, usando a visão de "aging". Ele faz isso porque precisamos recupearar a PMT, assumida contante, então assumimos que tomar o máximo vai funcionar como estimativa</p>
    """

    html_final = f"""
    <!DOCTYPE html>
    <html lang="pt-BR">
    <head>
        <meta charset="UTF-8">
        <title>Relatório de Performance de Vencimentos (Aging)</title>
        {html_css}
    </head>
    <body>
        <div class="main-container">
            <div class="report-header">
                <h1>Relatório de Performance de Vencimentos</h1>
                <p>Data de referência dos dados: {ref_date_obj.strftime('%d/%m/%Y')}</p>
            </div>
            {''.join(html_tabelas)}
            
            <footer>
                {footer_text}
            </footer>
        </div>
    </body>
    </html>
    """

    nome_arquivo_saida = f"{date.today().strftime('%Y-%m-%d')}-relatorio_performance_vencimentos_aging.html"
    caminho_arquivo_saida = os.path.join(output_path, nome_arquivo_saida)
    
    with open(caminho_arquivo_saida, "w", encoding="utf-8") as f:
        f.write(html_final)
    print("\n" + "="*80)
    print("PROCESSO FINALIZADO COM SUCESSO!")
    print(f"Relatório de Performance (Aging) gerado em: {caminho_arquivo_saida}")
    print("="*80)

else:
    print("\nNenhuma tabela foi gerada. O relatório final está vazio.")




INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS
Data de referência para os dados: 05/08/2025
---> Processando segmento: Carteira Consolidada...
     Pico de Valor Nominal para este segmento: R$ 1,678,416.74


C:\Users\Leo\AppData\Local\Temp\ipykernel_33772\1593312765.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Cedentes...
     [INFO] Segmento 'Cedentes' tem apenas uma entrada: 'BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A'. Gerando mensagem simples.
---> Processando segmento: Originadores...
     [INFO] Segmento 'Originadores' tem apenas uma entrada: 'StarCard'. Gerando mensagem simples.
---> Processando segmento: Promotoras...
     Pico de Valor Nominal para este segmento: R$ 258,210.27
---> Processando segmento: Produtos...


C:\Users\Leo\AppData\Local\Temp\ipykernel_33772\1593312765.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


     Pico de Valor Nominal para este segmento: R$ 469,550.56
---> Processando segmento: Convênios...


...

     Pico de Valor Nominal para este segmento: R$ 1,097,067.85
---> Processando segmento: CAPAG...
     Pico de Valor Nominal para este segmento: R$ 1,309,816.98


C:\Users\Leo\AppData\Local\Temp\ipykernel_33772\1593312765.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)
C:\Users\Leo\AppData\Local\Temp\ipykernel_33772\1593312765.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Pagamento Parcial...
     Pico de Valor Nominal para este segmento: R$ 1,678,416.74
---> Processando segmento: Tem Muitos Contratos...


C:\Users\Leo\AppData\Local\Temp\ipykernel_33772\1593312765.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)
C:\Users\Leo\AppData\Local\Temp\ipykernel_33772\1593312765.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


     Pico de Valor Nominal para este segmento: R$ 1,288,457.76
---> Processando segmento: Tem Muitos Entes...
     [INFO] Segmento 'Tem Muitos Entes' tem apenas uma entrada: 'False'. Gerando mensagem simples.

PROCESSO FINALIZADO COM SUCESSO!
Relatório de Performance (Aging) gerado em: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output\2025-08-12-relatorio_performance_vencimentos_aging.html


In [ ]:
#%%
# =============================================================================
# CÉLULA DE GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS (AGING)
# =============================================================================
# Descrição: Este script utiliza a lógica de análise de performance baseada no
# "mês de pico" de CADA LINHA. Ele calcula o volume nominal de cada mês como
# um percentual do mês de maior volume da história para aquele item específico.

import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

# --- CONFIGURAÇÕES INICIAIS ---

# Verifica se o DataFrame principal (df_final2) existe.
if 'df_final2' not in locals() or df_final2.empty:
    raise NameError("O DataFrame 'df_final2' não foi encontrado ou está vazio. Por favor, execute as células anteriores primeiro.")

# Adiciona uma coluna para permitir a análise da carteira total
df_report = df_final2.copy()
df_report['_total_carteira'] = 'Carteira Consolidada'
df_report['MesVencimento'] = pd.to_datetime(df_report['DataVencimento']).dt.to_period('M')

# Define os segmentos que queremos analisar
segmentos_para_analise = {
    'Carteira Consolidada': '_total_carteira',
    'Cedentes': 'Cedente',
    'Originadores': 'Originador',
    'Promotoras': 'Promotora',
    'Produtos': 'Produto',
    'Convênios': 'Convênio',
    'Situação': 'Situacao',
    'UF': 'UF',
    'CAPAG': 'CAPAG',
    'Pagamento Parcial': 'PagamentoParcial',
    'Tem Muitos Contratos':'_MuitosContratos',
    'Tem Muitos Entes':'_MuitosEntes'
}

# Define a data de referência
ref_date_obj = df_report['DataGeracao'].max().date()

# Define o caminho de saída
output_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output'
os.makedirs(output_path, exist_ok=True)


# --- FUNÇÕES AUXILIARES ---

def convert_period_to_relative_month(period: pd.Period, ref_date: date) -> str:
    """Converte um objeto Period para um formato de mês relativo (ex: M-1, M+0)."""
    venc_month = period.to_timestamp().to_pydatetime()
    ref_month = datetime(ref_date.year, ref_date.month, 1)
    delta = relativedelta(venc_month, ref_month)
    months_diff = delta.years * 12 + delta.months
    return f"M{months_diff:+}"

# --- FUNÇÃO PRINCIPAL DE GERAÇÃO DA TABELA ---

def generate_performance_heatmap(df_input: pd.DataFrame, segment_column: str, segment_friendly_name: str, ref_date: date) -> str:
    """
    Gera uma tabela HTML de performance de vencimentos para um segmento.
    """
    print(f"---> Processando segmento: {segment_friendly_name}...")

    # Lógica para pular segmentos com apenas uma entrada
    unique_entries = df_input[segment_column].dropna().unique()
    if len(unique_entries) == 1 and segment_column != '_total_carteira':
        entry_name = unique_entries[0]
        print(f"     [INFO] Segmento '{segment_friendly_name}' tem apenas uma entrada: '{entry_name}'. Gerando mensagem simples.")
        message_html = f"""
        <div class="table-container">
            <h3 class="table-title">Análise por: {segment_friendly_name}</h3>
            <p class="single-entry-message">Em '{segment_friendly_name}' temos apenas uma entrada: <strong>{entry_name}</strong>. A análise detalhada deste item é refletida na tabela 'Carteira Consolidada'.</p>
        </div>
        """
        return message_html

    # 1. Criação da Tabela Dinâmica com Valor Nominal
    report_abs = df_input.pivot_table(
        index=segment_column,
        columns="MesVencimento",
        values="ValorNominal",
        fill_value=0,
        aggfunc='sum'
    )
    if report_abs.empty: return ""

    # 2. <-- ALTERAÇÃO AQUI: Cálculo de Pico por Linha (Row-wise)
    # Em vez de um pico global, calculamos o pico para cada item do índice.
    # peak_value = report_abs.max().max() # <-- LÓGICA ANTIGA REMOVIDA
    row_peaks = report_abs.max(axis=1) # Encontra o valor máximo de cada linha
    # Lida com casos onde a linha inteira pode ser zero para evitar divisão por zero.
    row_peaks[row_peaks == 0] = np.nan

    # 3. Cálculo dos Totais de VP e VL
    summary_totals = df_input.groupby(segment_column)[['_ValorLiquido', 'ValorPresente']].sum()
    summary_totals.rename(columns={'_ValorLiquido': 'Valor Líquido Total (R$)', 'ValorPresente': 'Valor Presente Total (R$)'}, inplace=True)

    # 4. <-- ALTERAÇÃO AQUI: Cálculo da Performance Normalizado pela Linha
    # A divisão agora é feita linha por linha, usando o pico da respectiva linha.
    # O método .div com axis=0 garante que cada linha de report_abs seja dividida pelo seu pico correspondente em row_peaks.
    report_performance = (1 - report_abs.div(row_peaks, axis=0)) * 100
    report_performance["Total Nominal (R$)"] = report_abs.sum(axis=1)

    # Junta os totais de VP e VL
    report_performance = report_performance.join(summary_totals)

    report_performance.sort_values(by="Total Nominal (R$)", ascending=False, inplace=True)
    report_performance.index.name = segment_friendly_name
    report_performance.reset_index(inplace=True)

    # 5. Lógica de Renomeação e Ordenação de Colunas
    period_cols = [c for c in report_performance.columns if isinstance(c, pd.Period)]
    period_to_relative_map = {p: convert_period_to_relative_month(p, ref_date) for p in period_cols}
    relative_to_html_map = {
        relative: f"{relative}<br><span class='sub-header'>{period.strftime('%Y-%m')}</span>"
        for period, relative in period_to_relative_map.items()
    }
    report_performance.rename(columns=period_to_relative_map, inplace=True)

    relative_month_cols = [c for c in report_performance.columns if isinstance(c, str) and c.startswith('M')]

    # [ALTERADO] Inclui o M+1 na análise para ver adiantamentos
    months_to_show = [c for c in relative_month_cols if int(c.replace('M','').replace('+','')) <= 1]

    sort_key = lambda col: int(col.replace('M', '').replace('+', ''))
    # Inverte a ordem das colunas de mês
    sorted_relative_cols = sorted(months_to_show, key=sort_key, reverse=True)

    # Adiciona as colunas de VP e VL no início da ordem
    final_cols_order = [
        segment_friendly_name,
        'Valor Presente Total (R$)',
        'Valor Líquido Total (R$)'
    ] + sorted_relative_cols + ["Total Nominal (R$)"]

    # Garante que apenas colunas existentes sejam selecionadas
    final_cols_order = [col for col in final_cols_order if col in report_performance.columns]
    report_final = report_performance[final_cols_order]

    report_final.rename(columns=relative_to_html_map, inplace=True)
    sorted_html_cols = [relative_to_html_map.get(c, c) for c in sorted_relative_cols]

    # 6. Estilização e Geração do HTML
    def format_br_percent(val):
        if pd.isna(val): return ""
        return f"{val:.2f}%".replace('.', ',')

    def format_br_number(val):
        if pd.isna(val): return ""
        return f"{val:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')

    styler = (
        report_final.style
        .set_table_attributes('class="dataframe"')
        .set_properties(**{"padding": "8px", "text-align": "right", "min-width": "80px"})
        .set_table_styles([
            {"selector": "th, td", "props": [("border", "1px solid #ccc")]},
            {"selector": "th", "props": [("background-color", "#163f3f"), ("color", "#FFFFFF"), ("text-align", "center"), ("font-weight", "bold"), ("line-height", "1.3")]},
            {"selector": "thead th", "props": [("position", "sticky"), ("top", "0"), ("z-index", "1")]},
            {"selector": "tbody td:first-child", "props": [("position", "sticky"), ("left", "0"), ("background-color", "#f9f9f9"), ("text-align", "left"), ("font-weight", "bold"), ("white-space", "nowrap")]},
            {"selector": "thead th:first-child", "props": [("position", "sticky"), ("left", "0"), ("z-index", "2")]}
        ], overwrite=False)
        .format({
            **{col: format_br_percent for col in sorted_html_cols},
            **{"Total Nominal (R$)": format_br_number,
               'Valor Presente Total (R$)': format_br_number,
               'Valor Líquido Total (R$)': format_br_number}
        })
        .background_gradient(cmap="RdYlGn", subset=sorted_html_cols, vmin=0, vmax=100, axis=None)
        .background_gradient(cmap="Greys", subset=["Total Nominal (R$)", 'Valor Presente Total (R$)', 'Valor Líquido Total (R$)'], axis=None)
        .hide(axis="index")
    )

    header_tabela = f"""
    <div class="table-container">
        <h3 class="table-title">Performance de Vencimentos por: {segment_friendly_name}</h3>
        {styler.to_html()}
    </div>
    """
    return header_tabela


# --- BLOCO PRINCIPAL DE EXECUÇÃO ---

print("\n" + "="*80)
print("INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS")
print(f"Data de referência para os dados: {ref_date_obj.strftime('%d/%m/%Y')}")
print("="*80)

html_tabelas = []
for nome_amigavel, nome_coluna in segmentos_para_analise.items():
    tabela_html = generate_performance_heatmap(df_report, nome_coluna, nome_amigavel, ref_date_obj)
    if tabela_html:
        html_tabelas.append(tabela_html)

# --- MONTAGEM DO ARQUIVO HTML FINAL ---

if html_tabelas:
    html_css = """
    <style>
        body { font-family: "Segoe UI", Arial, sans-serif; background-color: #f4f4f4; margin: 0; padding: 20px; }
        .main-container { max-width: 95%; margin: auto; background-color: #ffffff; padding: 25px; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
        .report-header { text-align: center; border-bottom: 2px solid #163f3f; padding-bottom: 15px; margin-bottom: 30px; }
        .report-header h1 { color: #163f3f; margin: 0; }
        .report-header p { color: #555; font-size: 1.1em; }
        .table-container { margin-bottom: 50px; }
        .table-title { color: #0e5d5f; border-bottom: 2px solid #76c6c5; padding-bottom: 8px; margin-bottom: 15px; }
        .dataframe { border-collapse: collapse; border-spacing: 0; width: 100%; font-size: 0.9em; }
        .dataframe td.na { background-color: transparent !important; }
        .single-entry-message {
            padding: 20px;
            background-color: #f8f9fa;
            border-left: 5px solid #76c6c5;
            margin-top: 15px;
            font-size: 1.1em;
            color: #333;
        }
        .sub-header {
            font-size: 0.8em;
            font-weight: normal;
            color: #dddddd;
        }
        footer {
            margin-top: 40px;
            padding-top: 20px;
            border-top: 1px solid #ddd;
            font-size: 0.9em;
            color: #777;
            text-align: center;
        }
    </style>
    """

    # <-- ALTERAÇÃO AQUI: O texto do rodapé foi atualizado para descrever a nova metodologia.
    footer_text = """
    <p><strong>Nota sobre a Metodologia:</strong> Este relatório analisa a performance de cada item (ex: cada Originador) de forma individual.
    Para cada linha da tabela, o relatório identifica o mês de maior 'Valor Nominal' ('mês de pico' daquela linha) e o usa como referência (100%).
    Os demais meses da mesma linha são então mostrados como um percentual desse pico pessoal. Isso permite avaliar a consistência e a tendência de cada item em relação ao seu próprio melhor desempenho histórico.</p>
    """

    html_final = f"""
    <!DOCTYPE html>
    <html lang="pt-BR">
    <head>
        <meta charset="UTF-8">
        <title>Relatório de Performance de Vencimentos (Aging)</title>
        {html_css}
    </head>
    <body>
        <div class="main-container">
            <div class="report-header">
                <h1>Relatório de Performance de Vencimentos</h1>
                <p>Data de referência dos dados: {ref_date_obj.strftime('%d/%m/%Y')}</p>
            </div>
            {''.join(html_tabelas)}

            <footer>
                {footer_text}
            </footer>
        </div>
    </body>
    </html>
    """

    nome_arquivo_saida = f"{date.today().strftime('%Y-%m-%d')}-relatorio_performance_vencimentos_aging.html"
    caminho_arquivo_saida = os.path.join(output_path, nome_arquivo_saida)

    with open(caminho_arquivo_saida, "w", encoding="utf-8") as f:
        f.write(html_final)
    print("\n" + "="*80)
    print("PROCESSO FINALIZADO COM SUCESSO!")
    print(f"Relatório de Performance (Aging) gerado em: {caminho_arquivo_saida}")
    print("="*80)

else:
    print("\nNenhuma tabela foi gerada. O relatório final está vazio.")